## Redes sociais - APS(1)

Diogo Duarte, Pedro Dannecker e Gabriel de Araújo Alves



### Importando bibliotecas

In [5]:
import pandas as pd
import csv
from graph_tool import Graph
import os
from graph_tool import util
# from netpixi.data.gt import *
# import netpixi

### Analisando os dados iniciais

### Criando o grafo

In [6]:
from netpixi.integration.gt import Graph

g = Graph(directed=False) # não-dirigido

### Analise exploratoria

#### a)número de vértices, número de arestas, densidade;

#### b)estatísticas e histograma do degree. (não é necessário in-degree e out-degree)

### Análise visual do grafo:











#### a)salve o grafo em um arquivo;

#### b)visualize o grafo a partir desse arquivo;

#### c)posicione os vértices de acordo com um algoritmo baseado em force-directed graph drawing, para evidenciar grupos; (pode ser qualquer um deles)

#### d)ajuste as propriedades visuais do grafo para deixá-lo o mais claro possível; (por exemplo, se a visualização é simplesmente uma massa de vértices que cobre todas as arestas, reduza os vértices para tentar deixar as arestas mais visíveis)


#### e)exporte o grafo para uma imagem PNG. (há um botão para isso)
